In [1]:
using DifferentiableFactorizations, LinearAlgebra, BenchmarkTools

In [2]:
A = rand(5, 3)

5×3 Matrix{Float64}:
 0.845842  0.336823  0.357544
 0.613874  0.240528  0.26055
 0.466771  0.555022  0.839431
 0.950547  0.848541  0.661852
 0.215324  0.332352  0.458818

In [3]:
x = diff_qr(A)
(; Q, R) = x

(Q = [-0.5626374246916124 -0.534996307457174 -0.1911014807447391; -0.4083366324798291 -0.3966473451485766 -0.15515095867580989; … ; -0.6322846838906959 0.4019384118687258 0.6553688308465081; -0.14322894073275036 0.39013799552116596 -0.25072771501405505], R = [-1.503352449296859 -1.0441742589767813 -1.0523860311291044; 0.0 0.46854259144961447 0.5639264134096169; 0.0 0.0 -0.35128271153871443])

In [19]:
@benchmark begin
    vcat(
        vec(UpperTriangular(Q' * Q) + LowerTriangular(R) - I - Diagonal(R)),
        vec(Q * R - A),
    )
end

  1.995 μs (18 allocations: 1.73 KiB)


24-element Vector{Float64}:
  0.0
  0.0
  0.0
  2.2744458677460948e-17
 -7.216449660063518e-16
  0.0
 -1.0063670186567891e-16
 -1.0261983105514117e-16
  3.885780586188048e-16
  3.3306690738754696e-16
  ⋮
  1.3877787807814457e-16
  0.0
  1.1102230246251565e-16
  0.0
  1.6653345369377348e-16
  4.440892098500626e-16
 -2.220446049250313e-16
  0.0
 -1.6653345369377348e-16

In [18]:
@benchmark begin
  m, n = size(A, 1), size(A, 2)
  res = similar(A, m + n, n)
  res_A = view(res, 1:m, :)
  res_R = view(res, m+1:m+n, :)
  copyto!(res_A, A)
  fill!(res_R, 0)
  for i in 1:min(m, n)
    res_R[i, i] = 1
  end
  for i in 1:m
    for j in 1:n
      for k in 1:n
        res_A[i, j] -= Q[i, k] * R[k, j]
        if i == 1 && j < k
          res_R[k, j] += R[k, j]
        end
        if j <= k
          res_R[j, k] -= Q[i, k] * Q[i, j]
        end
      end
    end
  end
  vec(res)
end

  16.774 μs (488 allocations: 9.58 KiB)


24-element Vector{Float64}:
 -3.3306690738754696e-16
  0.0
  0.0
  0.0
  0.0
 -1.3530843112619095e-16
  0.0
  0.0
 -1.6653345369377348e-16
 -1.3877787807814457e-16
  ⋮
  0.0
 -1.6653345369377348e-16
 -4.2327252813834093e-16
  2.220446049250313e-16
  5.551115123125783e-17
  1.6653345369377348e-16
  9.020562075079397e-17
  9.71445146547012e-17
 -3.885780586188048e-16